In [1]:
import os
import glob
import findspark
import pandas as pd
findspark.init()
findspark.find()

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T

!python --version

Python 3.11.6


In [2]:

spark_version = '3.4.1'
os.environ['PYSPARK_SUBMIT_ARGS'] = f'--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:{spark_version},org.apache.spark:spark-sql-kafka-0-10_2.12:{spark_version} pyspark-shell'


In [3]:
from pyspark.sql import SparkSession
session = SparkSession.builder.appName('ENG').config('hive.exec.dynamic.partition.mode', 'nonstrict').config("spark.memory.offHeap.size","4g").enableHiveSupport().getOrCreate()

In [4]:
session

In [4]:
df = pd.read_csv("datasets/Pokemon_full.csv")

In [5]:
df1 = df[df.type.isin(['grass', 'fire', 'dragon', 'fairy'])]
df2 = df[~df.type.isin(['grass', 'fire', 'dragon', 'fairy'])]

In [6]:
data1 = session.createDataFrame(df1)
data2 = session.createDataFrame(df2)

/usr/local/spark/python/pyspark/sql/pandas/conversion.py:479: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:


In [7]:
for col, new_col in zip(data1.columns, [x.replace(' ', '_') for x in data1.columns]):
    data1 = data1.withColumnRenamed(col, new_col)

data1.printSchema()

for col, new_col in zip(data2.columns, [x.replace(' ', '_') for x in data2.columns]):
    data2 = data2.withColumnRenamed(col, new_col)

data2.printSchema()

root
 |-- name: string (nullable = true)
 |-- pokedex_id: long (nullable = true)
 |-- height: long (nullable = true)
 |-- weight: long (nullable = true)
 |-- type: string (nullable = true)
 |-- secundary_type: string (nullable = true)
 |-- hp: long (nullable = true)
 |-- attack: long (nullable = true)
 |-- defense: long (nullable = true)
 |-- sp_atk: long (nullable = true)
 |-- sp_def: long (nullable = true)
 |-- speed: long (nullable = true)

root
 |-- name: string (nullable = true)
 |-- pokedex_id: long (nullable = true)
 |-- height: long (nullable = true)
 |-- weight: long (nullable = true)
 |-- type: string (nullable = true)
 |-- secundary_type: string (nullable = true)
 |-- hp: long (nullable = true)
 |-- attack: long (nullable = true)
 |-- defense: long (nullable = true)
 |-- sp_atk: long (nullable = true)
 |-- sp_def: long (nullable = true)
 |-- speed: long (nullable = true)



In [13]:
session.sql('drop table if exists pokemon')

DataFrame[]

## Partitioned table

In [14]:
q = '''create external table if not exists pokemon(
name string,
pokedex_id long,
height long, 
weight long,
secundary_type string, 
hp long, 
attack long, 
defense long,
sp_atk long,
sp_def long,
speed long
)
PARTITIONED by (type string)
STORED AS PARQUET
LOCATION 'pokemon'
'''

In [15]:
session.sql(q)

DataFrame[]

In [16]:
session.table('pokemon').show()

+----+----------+------+------+--------------+---+------+-------+------+------+-----+----+
|name|pokedex_id|height|weight|secundary_type| hp|attack|defense|sp_atk|sp_def|speed|type|
+----+----------+------+------+--------------+---+------+-------+------+------+-----+----+
+----+----------+------+------+--------------+---+------+-------+------+------+-----+----+



In [32]:
#data1 = data1.withColumn('height', F.col('height').cast(T.StringType())).withColumn('weight', F.col('weight').cast(T.StringType()))

In [17]:
data1 = data1.withColumn('height', F.col('height').cast(T.IntegerType())).withColumn('weight', F.col('weight').cast(T.IntegerType()))

In [18]:
data1.select(session.table('pokemon').columns).write.format('parquet').mode("append").insertInto('pokemon')

In [19]:
session.table('pokemon').groupby('type').agg(F.count('name').alias('records')).show()

+------+-------+
|  type|records|
+------+-------+
| grass|     86|
|  fire|     58|
|dragon|     30|
| fairy|     21|
+------+-------+



In [22]:
# First insert
data2.select(session.table('pokemon').columns).write.format('parquet').mode("append").insertInto('pokemon')
session.table('pokemon').groupby('type').agg(F.count('name').alias('records')).show()

+--------+-------+
|    type|records|
+--------+-------+
|  normal|    218|
|     bug|    150|
|   grass|     86|
|   water|    246|
| psychic|    114|
|electric|     96|
|     ice|     54|
|    rock|    100|
|    fire|     58|
|   steel|     60|
|fighting|     68|
|  poison|     70|
|   ghost|     60|
|    dark|     70|
|  dragon|     30|
|  ground|     70|
|   fairy|     21|
|  flying|     14|
+--------+-------+



In [21]:
# Second insert
data2.select(session.table('pokemon').columns).write.format('parquet').mode("append").insertInto('pokemon')
session.table('pokemon').groupby('type').agg(F.count('name').alias('records')).show()

+--------+-------+
|    type|records|
+--------+-------+
|  normal|    218|
|     bug|    150|
|   grass|    172|
|   water|    246|
|electric|     96|
|   steel|     60|
|    dark|     70|
|  poison|     70|
| psychic|    114|
|    rock|    100|
|    fire|    116|
|fighting|     68|
|  dragon|     60|
|   ghost|     60|
|     ice|     54|
|  ground|     70|
|   fairy|     42|
|  flying|     14|
+--------+-------+



In [23]:
# Insert overwrite
data2.filter('height < 10').select(session.table('pokemon').columns).write.format('parquet').mode("overwrite").insertInto('pokemon')
session.table('pokemon').groupby('type').agg(F.count('name').alias('records')).show()

+--------+-------+
|    type|records|
+--------+-------+
|     bug|     43|
|  normal|     56|
|   grass|     86|
|  dragon|     30|
|    fire|     58|
| psychic|     34|
|electric|     26|
|   ghost|     15|
|   water|     58|
|   fairy|     21|
|   steel|     14|
|     ice|      9|
|  poison|     17|
|    rock|     19|
|  flying|      4|
|    dark|     16|
|fighting|     13|
|  ground|     12|
+--------+-------+



## Non-partitioned table

In [34]:
q = '''create external table if not exists pokemon_nonpart(
name string,
pokedex_id long,
height long, 
weight long,
secundary_type string, 
hp long, 
attack long, 
defense long,
sp_atk long,
sp_def long,
speed long,
type string
)
STORED AS PARQUET
LOCATION 'pokemon_nonpart'
'''
session.sql('drop table if exists pokemon_nonpart')
session.sql(q)

DataFrame[]

In [36]:
data1.select(session.table('pokemon_nonpart').columns).write.format('parquet').mode("overwrite").insertInto('pokemon_nonpart')
session.table('pokemon_nonpart').groupby('type').agg(F.count('name').alias('records')).show()

+------+-------+
|  type|records|
+------+-------+
| grass|     86|
| fairy|     21|
|dragon|     30|
|  fire|     58|
+------+-------+



In [37]:
data2.select(session.table('pokemon_nonpart').columns).write.format('parquet').mode("overwrite").insertInto('pokemon_nonpart')
session.table('pokemon_nonpart').groupby('type').agg(F.count('name').alias('records')).show()

+--------+-------+
|    type|records|
+--------+-------+
|   ghost|     30|
|   steel|     30|
|     ice|     27|
|   water|    123|
|  ground|     35|
|  flying|      7|
|    dark|     35|
|fighting|     34|
|  poison|     35|
| psychic|     57|
|    rock|     50|
|electric|     48|
|  normal|    109|
|     bug|     75|
+--------+-------+



### Adding new columns

In [91]:
session.sql('drop table if exists pokemon')

DataFrame[]

In [24]:
q = '''create external table if not exists pokemon(
name string,
pokedex_id long,
height long, 
weight long,
secundary_type string, 
hp long, 
attack long, 
defense long,
sp_atk long,
sp_def long,
speed long,
new_test_column string,
new_another_column long
)
PARTITIONED by (type string)
STORED AS PARQUET
LOCATION 'pokemon'
'''



In [25]:
session.sql(q)

DataFrame[]

In [26]:
session.sql('MSCK REPAIR TABLE pokemon')

DataFrame[]

In [95]:
session.table('pokemon').show()

+----------+----------+------+------+--------------+---+------+-------+------+------+-----+---------------+------------------+-----+
|      name|pokedex_id|height|weight|secundary_type| hp|attack|defense|sp_atk|sp_def|speed|new_test_column|new_another_column| type|
+----------+----------+------+------+--------------+---+------+-------+------+------+-----+---------------+------------------+-----+
|    horsea|       116|     4|    80|          None| 30|    40|     70|    70|    25|   60|           null|              null|water|
|    seadra|       117|    12|   250|          None| 55|    65|     95|    95|    45|   85|           null|              null|water|
|   goldeen|       118|     6|   150|          None| 45|    67|     60|    35|    50|   63|           null|              null|water|
|   seaking|       119|    13|   390|          None| 80|    92|     65|    65|    80|   68|           null|              null|water|
|    staryu|       120|     8|   345|          None| 30|    45|     5

### Deleting columns

In [96]:
session.sql('drop table if exists pokemon')

DataFrame[]

In [97]:
q = '''create external table if not exists pokemon(
name string,
pokedex_id long,
height long, 
weight long,
secundary_type string, 
hp long, 
sp_atk long,
sp_def long,
speed long,
new_test_column string,
new_another_column long
)
PARTITIONED by (type string)
STORED AS PARQUET
LOCATION 'pokemon'
'''



In [98]:
session.sql(q)

DataFrame[]

In [99]:
session.sql('MSCK REPAIR TABLE pokemon')

DataFrame[]

In [100]:
session.table('pokemon').show()

+----------+----------+------+------+--------------+---+------+------+-----+---------------+------------------+-----+
|      name|pokedex_id|height|weight|secundary_type| hp|sp_atk|sp_def|speed|new_test_column|new_another_column| type|
+----------+----------+------+------+--------------+---+------+------+-----+---------------+------------------+-----+
|    horsea|       116|     4|    80|          None| 30|    70|    25|   60|           null|              null|water|
|    seadra|       117|    12|   250|          None| 55|    95|    45|   85|           null|              null|water|
|   goldeen|       118|     6|   150|          None| 45|    35|    50|   63|           null|              null|water|
|   seaking|       119|    13|   390|          None| 80|    65|    80|   68|           null|              null|water|
|    staryu|       120|     8|   345|          None| 30|    70|    55|   85|           null|              null|water|
|   starmie|       121|    11|   800|       psychic| 60|

In [105]:
data3 = (data1
         .withColumn('type', F.concat(F.col('type'), F.lit('_new')))
        .withColumn('new_test_column', F.lit(1))
        .withColumn('new_another_column', F.lit(2))
        )

In [107]:
data3.select(session.table('pokemon').columns).write.format('parquet').mode("append").insertInto('pokemon')

In [108]:
session.table('pokemon').show()

+----------+----------+------+------+--------------+---+------+------+-----+---------------+------------------+-----+
|      name|pokedex_id|height|weight|secundary_type| hp|sp_atk|sp_def|speed|new_test_column|new_another_column| type|
+----------+----------+------+------+--------------+---+------+------+-----+---------------+------------------+-----+
|    horsea|       116|     4|    80|          None| 30|    70|    25|   60|           null|              null|water|
|    seadra|       117|    12|   250|          None| 55|    95|    45|   85|           null|              null|water|
|   goldeen|       118|     6|   150|          None| 45|    35|    50|   63|           null|              null|water|
|   seaking|       119|    13|   390|          None| 80|    65|    80|   68|           null|              null|water|
|    staryu|       120|     8|   345|          None| 30|    70|    55|   85|           null|              null|water|
|   starmie|       121|    11|   800|       psychic| 60|

In [109]:
session.table('pokemon').groupby('type').agg(F.count('name').alias('records')).show()

+----------+-------+
|      type|records|
+----------+-------+
| grass_new|     86|
|    normal|    109|
|       bug|     75|
|     grass|     86|
|     water|    123|
|      dark|     35|
|   psychic|     57|
|  electric|     48|
|     steel|     30|
|    poison|     35|
|      rock|     50|
|      fire|     58|
|  fighting|     34|
|    dragon|     30|
|dragon_new|     30|
|  fire_new|     58|
|     ghost|     30|
|       ice|     27|
|    ground|     35|
| fairy_new|     21|
+----------+-------+
only showing top 20 rows



In [8]:
### SIZE OF DATAFRAME ESTIMATED ###

raw_size = session._jvm.org.apache.spark.util.SizeEstimator.estimate(session.table('pokemon')._jdf)
mb_size = raw_size / (1024*1024)
mb_size

22.36180877685547

## Working with FileSystem

In [12]:
fs = (session._jvm.org
      .apache.hadoop
      .fs.FileSystem
      .get(session._jsc.hadoopConfiguration())
      )

fs.exists(session._jvm.org.apache.hadoop.fs.Path("./spark-warehouse/pokemon/"))

True

In [14]:
path = "./spark-warehouse/pokemon/"
fs = session._jvm.org.apache.hadoop.fs.FileSystem.get(session._jsc.hadoopConfiguration())
list_status = fs.listStatus(session._jvm.org.apache.hadoop.fs.Path(path))
result = [file.getPath().getName() for file in list_status]

result

['type=bug',
 'type=dark',
 'type=dragon',
 'type=electric',
 'type=fairy',
 'type=fighting',
 'type=fire',
 'type=flying',
 'type=ghost',
 'type=grass',
 'type=ground',
 'type=ice',
 'type=normal',
 'type=poison',
 'type=psychic',
 'type=rock',
 'type=steel',
 'type=water',
 '_SUCCESS']

In [15]:
sc = session

hadoop = sc._jvm.org.apache.hadoop

fs = hadoop.fs.FileSystem
conf = hadoop.conf.Configuration() 
path = hadoop.fs.Path("./spark-warehouse/pokemon/")

for f in fs.get(conf).listStatus(path):
    print(f.getPath(), f.getLen())

file:/home/jovyan/work/spark-warehouse/pokemon/type=bug 512
file:/home/jovyan/work/spark-warehouse/pokemon/type=dark 512
file:/home/jovyan/work/spark-warehouse/pokemon/type=dragon 512
file:/home/jovyan/work/spark-warehouse/pokemon/type=electric 512
file:/home/jovyan/work/spark-warehouse/pokemon/type=fairy 512
file:/home/jovyan/work/spark-warehouse/pokemon/type=fighting 512
file:/home/jovyan/work/spark-warehouse/pokemon/type=fire 512
file:/home/jovyan/work/spark-warehouse/pokemon/type=flying 512
file:/home/jovyan/work/spark-warehouse/pokemon/type=ghost 512
file:/home/jovyan/work/spark-warehouse/pokemon/type=grass 512
file:/home/jovyan/work/spark-warehouse/pokemon/type=ground 512
file:/home/jovyan/work/spark-warehouse/pokemon/type=ice 512
file:/home/jovyan/work/spark-warehouse/pokemon/type=normal 512
file:/home/jovyan/work/spark-warehouse/pokemon/type=poison 512
file:/home/jovyan/work/spark-warehouse/pokemon/type=psychic 512
file:/home/jovyan/work/spark-warehouse/pokemon/type=rock 512
fi

In [23]:
import time

fs = session._jvm.org.apache.hadoop.fs.FileSystem.get(session._jsc.hadoopConfiguration())
path =  session._jvm.org.apache.hadoop.fs.Path("./spark-warehouse/pokemon/")

time_ts = fs.getFileStatus(path).getModificationTime()
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(time_ts / 1000))) 

2023-11-11 15:01:40


In [144]:
FS = fs.get(conf)
FS.delete(hadoop.fs.Path('/pokemon/type=rock/'), True) 

False

# Прочитать из топика

In [ ]:
from kafka import KafkaConsumer
import json

kafka_servers='broker_kafka:29092'
topic= 'WriteTopic'

consumer = KafkaConsumer(topic,bootstrap_servers=kafka_servers)
#print(consumer.config)
#print(consumer.bootstrap_connected())

for msg in consumer:

    payload = json.loads(msg.value)
    payload["meta_data"]={
        "topic":msg.topic,
        "partition":msg.partition,
        "offset":msg.offset,
        "timestamp":msg.timestamp,
        "timestamp_type":msg.timestamp_type,
        "key":msg.key,
    }

    print(payload)

{'header': {'send_at': '2024-03-08 19:13:20Z'}, 'body': {'Тип': 'bug', 'доп. тип': 'NaN', 'метрики': {'метрики': ['{"покедекс_ид":736,"name":"grubbin","hp":47,"attack":62,"defense":45}', '{"покедекс_ид":824,"name":"blipbug","hp":25,"attack":20,"defense":20}', '{"покедекс_ид":10,"name":"caterpie","hp":45,"attack":30,"defense":35}', '{"покедекс_ид":11,"name":"metapod","hp":50,"attack":20,"defense":55}', '{"покедекс_ид":127,"name":"pinsir","hp":65,"attack":125,"defense":100}', '{"покедекс_ид":204,"name":"pineco","hp":50,"attack":65,"defense":90}', '{"покедекс_ид":265,"name":"wurmple","hp":45,"attack":45,"defense":35}', '{"покедекс_ид":266,"name":"silcoon","hp":50,"attack":35,"defense":55}', '{"покедекс_ид":268,"name":"cascoon","hp":50,"attack":35,"defense":55}', '{"покедекс_ид":313,"name":"volbeat","hp":65,"attack":73,"defense":75}', '{"покедекс_ид":314,"name":"illumise","hp":65,"attack":47,"defense":75}', '{"покедекс_ид":401,"name":"kricketot","hp":37,"attack":25,"defense":41}', '{"покед

In [5]:
payload

{'header': {'send_at': '2024-03-08 18:57:42Z'},
 'body': {'Тип': 'water',
  'доп. тип': 'steel',
  'метрики': {'метрики': ['{"покедекс_ид":395,"name":"empoleon","hp":84,"attack":86,"defense":88}']}},
 'meta_data': {'topic': 'WriteTopic',
  'partition': 0,
  'offset': 2489,
  'timestamp': 1709924262886,
  'timestamp_type': 0,
  'key': None}}

In [5]:
kafka_servers='localhost:9092'
kafka_servers='broker_kafka:29092'
topic= 'WriteTopic'

In [7]:
stream = session.readStream.format("kafka") \
    .option("kafka.bootstrap.servers", kafka_servers) \
    .option("kafka.security.protocol", "PLAINTEXT") \
    .option("subscribe", topic) \
    .option("startingOffsets", "earliest") \
    .load()

In [10]:
stream\
    .writeStream \
    .format("console") \
    .option("truncate", "false") \
    .trigger(processingTime="5 second") \
    .start().awaitTermination()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 